In [ ]:
 script_info = """
 This notebook was created based on https://github.com/slychief/ismir2018_tutorial/blob/master/Part_3b_RNN_Onset_Detection.ipynb;
 There is a huge part of this code that is exactly copied; we just adapted their model to our case.
 Also, this is an EXAMPLE notebook, so it gets easier to understand how the model
 was created, the only one change that I made from the original one, is that
 I put n_epochs = 10 instead of 100 (the original had 100 epochs) for making
 this easier to preprocess. When I do the eval results, I do it with the original
 model which will located in ./models folder of this repo. 
 This is build on keras (tf as backend).
  """

In [1]:
!pip install madmom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.0 MB 1.3 MB/s 
     |████████████████████████████████| 51 kB 3.0 MB/s 
  Created wheel for madmom: filename=madmom-0.16.1-cp37-cp37m-linux_x86_64.whl size=20939879 sha256=78ad550a4ffcf25dbea3a9d2c7cbf4ced84208261431209814bee6aab7661d6f
  Stored in directory: /root/.cache/pip/wheels/af/90/61/393ceef814b55b12d1b59b5ed3a2b2a3457a55d39b7363b975
Successfully built madmom


In [2]:
from __future__ import print_function, division
import librosa
from google.colab import drive
import os
import sys
import glob
import logging
import warnings
import pickle
import matplotlib.pyplot as plt
# data processing / signal
import pandas as pd
import numpy as np
from sympy import Interval
import librosa
import re
# madmom
import madmom
from madmom.processors import ParallelProcessor, SequentialProcessor
from madmom.audio.signal import SignalProcessor, FramedSignalProcessor
from madmom.audio.stft import ShortTimeFourierTransformProcessor
from madmom.audio.spectrogram import FilteredSpectrogramProcessor, LogarithmicSpectrogramProcessor, SpectrogramDifferenceProcessor
# deeplearning
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Bidirectional, Masking, LSTM, Dense
from tensorflow.keras.utils import Sequence

# custom code

In [3]:
ROOT_DIR = "/content/drive"
drive.mount(ROOT_DIR, force_remount=True)
#Don't forget to type My Drive before the whole path
MUSIC_DIR = os.path.join(ROOT_DIR,'My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/')
AUDIO_DIR = ONSET_PATH = os.path.join(MUSIC_DIR,"audio","drum_only")
ANNOTATIONS_DIR = os.path.join(MUSIC_DIR,"annotations","class","train" )#/class/train"
example_suffix = "for_github"

Mounted at /content/drive


In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [5]:
FPS = 100
os.listdir(ANNOTATIONS_DIR)

['MusicDelta_Disco_class.txt',
 'MusicDelta_Britpop_class.txt',
 'MusicDelta_Rock_class.txt',
 'MusicDelta_BebopJazz_class.txt',
 'MusicDelta_Shadows_class.txt',
 'MusicDelta_Reggae_class.txt',
 'MusicDelta_CoolJazz_class.txt',
 'MusicDelta_Rockabilly_class.txt',
 'MusicDelta_FunkJazz_class.txt',
 'MusicDelta_FusionJazz_class.txt',
 'MusicDelta_80sRock_class.txt']

In [6]:
# custom code, need to perform mount operation before importing it
# you can find eval_utils script in the repo (./utils) ;
#  cd where your downnloaded eval_utils script is
%cd "/content/drive/My Drive/Colab Notebooks/tesis_esp"
from eval_utils import search_correspondingpath_given_annotation, load_labels_just_onsets

/content/drive/My Drive/Colab Notebooks/tesis_esp


In [8]:
class Dataset(object):
    def __init__(self, path, files, annotation_files, audio_files):
        self.path = path
        self.files, self.annotation_files, self.audio_files = files, annotation_files, audio_files 
    def load_splits(self, path=None, fold_suffix='.fold'):
        """
        We are NOT Using crossvalidation , so we weont use this function
        """
        path = path if path is not None else self.path + '/splits'
        self.split_files = madmom.utils.search_files(path, fold_suffix, recursion_depth=1)
        # populate folds
        self.folds = []
        for i, split_file in enumerate(self.split_files):
            fold_idx = []
            with open(split_file) as f:
                for file in f:
                    file = file.strip()
                    # get matching file idx
                    try:
                        idx = self.files.index(file)
                        fold_idx.append(idx)
                    except ValueError:
                        warnings.warn('no matching audio/annotation files: %s' % file)
                        continue
            # set indices for fold
            self.folds.append(np.array(fold_idx))
            
    def pre_process(self, pre_processor):
        self.x = [pre_processor(file) for file in self.audio_files]
        
    def load_annoatations(self, widen=None):
        "Function for loading all annotations"
        self.annotations = [load_labels_just_onsets(file) for file in self.annotation_files]

In [9]:
annotation_files_all = glob.glob(ANNOTATIONS_DIR+"/*.txt")
audio_files_all = [
              search_correspondingpath_given_annotation(annotation, ONSET_PATH)  
              for annotation in annotation_files_all
              ]
# take base path with  suffix and then remove it
files = [os.path.basename(file) for file in audio_files_all]
files = [file.split(".")[0] for file in files]
files


['MusicDelta_Disco_Drum',
 'MusicDelta_Britpop_Drum',
 'MusicDelta_Rock_Drum',
 'MusicDelta_BebopJazz_Drum',
 'MusicDelta_Shadows_Drum',
 'MusicDelta_Reggae_Drum',
 'MusicDelta_CoolJazz_Drum',
 'MusicDelta_Rockabilly_Drum',
 'MusicDelta_FunkJazz_Drum',
 'MusicDelta_FusionJazz_Drum',
 'MusicDelta_80sRock_Drum']

# 1) Load annotations into dataset obj

In [10]:
#  WARNING! file path MUST BE SORTED, when you pass the parameters
onsets_db = Dataset(path = ONSET_PATH,
                    files = files,
                    annotation_files = annotation_files_all,
                    audio_files= audio_files_all)
onsets_db.load_annoatations()
# first 5 onsets of the first file
onsets_db.annotations[0][:5]

[0.27, 0.273469, 0.41, 0.53, 0.68]

# 2) Preprocess the signal with 3 band filtered spectrogram differences:

In [11]:
# define pre-processor
class OnsetPreProcessor(SequentialProcessor):
  """
  Preprocessor of the raw signal: get the filtered spectroram differences with 3 band
  """
  def __init__(self, frame_sizes=[1024, 2048, 4096], num_bands=[3, 6, 12]):
      # resample to a fixed sample rate in order to get always the same number of filter bins
      sig = SignalProcessor(num_channels=1, sample_rate=44100)
      # process multi-resolution spec & diff in parallel
      multi = ParallelProcessor([])
      for frame_size, num_bands in zip(frame_sizes, num_bands):
          # split audio signal in overlapping frames
          frames = FramedSignalProcessor(frame_size=frame_size)
          # compute STFT
          stft = ShortTimeFourierTransformProcessor()
          # filter the magnitudes
          filt = FilteredSpectrogramProcessor(num_bands=num_bands)
          # scale them logarithmically
          spec = LogarithmicSpectrogramProcessor()
          # stack positive differences
          diff = SpectrogramDifferenceProcessor(positive_diffs=True, stack_diffs=np.hstack)
          # process each frame size with spec and diff sequentially
          multi.append(SequentialProcessor((frames, stft, filt, spec, diff)))
      # instantiate a SequentialProcessor
      super(OnsetPreProcessor, self).__init__((sig, multi, np.hstack))

# create a callable pre-processor
pp = OnsetPreProcessor()

## Drop the transformed signals into a pkl

In [12]:
path_to_dump_preprocess = os.path.join(MUSIC_DIR,f"onset_db_{example_suffix}.pkl")
if not os.path.exists(path_to_dump_preprocess):
  onsets_db.pre_process(pp)
  pickle.dump(onsets_db, open(path_to_dump_preprocess, 'wb'), protocol=2)
  print(f"Pickle dumped to: {path_to_dump_preprocess} ")
else:
  print(f"Loading previously dumped pickle from: {path_to_dump_preprocess}")
  onsets_db = pickle.load(open(path_to_dump_preprocess,'rb'))

Loading previously dumped pickle from: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/onset_db_for_github.pkl


In [13]:
onsets_db.x[0].shape

(12478, 314)

In [14]:
max2print = 5
for idx in range(len(onsets_db.annotation_files)):
  print("===================================",idx,"===================")
  print("Audiofile:",onsets_db.audio_files[idx])
  print("Annot:",onsets_db.annotation_files[idx])
  if idx>max2print:
    break

=================================== 0 ===================
Audiofile: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/train/MusicDelta_Disco_Drum.wav
Annot: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/annotations/class/train/MusicDelta_Disco_class.txt
=================================== 1 ===================
Audiofile: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/train/MusicDelta_Britpop_Drum.wav
Annot: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/annotations/class/train/MusicDelta_Britpop_class.txt
=================================== 2 ===================
Audiofile: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/train/MusicDelta_Rock_Drum.wav
Annot: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/annotations/class/train/M

# So each audio file has a lot of onsets within, and this can be proving by looking at each file with its corresponding annotations

In [15]:
onsets_db.x[1].shape

(3679, 314)

# 3) Preparing data for the rnn

In [16]:
class DataSequence(Sequence):
    mask_value = -999  # only needed for batch sizes > 1
    def __init__(self, x, y, batch_size=1, max_seq_length=None, fps=FPS):
        self.x = x
        # binarization
        self.y = [madmom.utils.quantize_events(o, fps=fps, length=len(d))
                  for o, d in zip(y, self.x)]
        self.batch_size = batch_size
        self.max_seq_length = max_seq_length
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        # determine which sequence(s) to use
        x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        # pad them if needed
        if self.batch_size > 1:
            x = keras.preprocessing.sequence.pad_sequences(
                x, maxlen=self.max_seq_length, dtype=np.float32, truncating='post', value=self.mask_value)
            y = keras.preprocessing.sequence.pad_sequences(
                y, maxlen=self.max_seq_length, dtype=np.int32, truncating='post', value=self.mask_value)
        return np.array(x), np.array(y)[..., np.newaxis]

In [17]:
basedir  = os.path.join(MUSIC_DIR,f"models_{example_suffix}")
if not os.path.exists(basedir ):
  os.mkdir(basedir )
  basedir  = os.path.join(basedir ,'onsets')
  if not os.path.exists(basedir ):
    os.mkdir(basedir)
    print("MKDIR FINISHED:",basedir )

In [18]:
basedir

'/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/models_for_github'

In [19]:
onsets_db.files

['MusicDelta_Disco_Drum',
 'MusicDelta_Britpop_Drum',
 'MusicDelta_Rock_Drum',
 'MusicDelta_BebopJazz_Drum',
 'MusicDelta_Shadows_Drum',
 'MusicDelta_Reggae_Drum',
 'MusicDelta_CoolJazz_Drum',
 'MusicDelta_Rockabilly_Drum',
 'MusicDelta_FunkJazz_Drum',
 'MusicDelta_FusionJazz_Drum',
 'MusicDelta_80sRock_Drum']

In [20]:
# we will select our validation set, notrandomly
all_files = onsets_db.files
validation_files = ["MusicDelta_Britpop_Drum", 'MusicDelta_Rock_Drum', 'MusicDelta_FunkJazz_Drum']
train_files = list( set(all_files) - set(validation_files) )
assert len(all_files) == (len(train_files) + len(validation_files)),"Error"

In [21]:
# now we retrieve the indexes
validation_files_indexes = [onsets_db.files.index(validation_files[val_idx]) for val_idx in range(len(validation_files))]
train_files_indexes = [onsets_db.files.index(train_files[train_idx]) for train_idx in range(len(train_files))]
validation_files_indexes

[1, 2, 8]

In [22]:
idx = 0
print(f"File name is {onsets_db.files[idx]}")
onset = onsets_db.x[idx]
annotation = onsets_db.annotations[idx]
print("Onset shape is",onset.shape)
quantize_onset = madmom.utils.quantize_events(annotation, fps=FPS, length = len(onset))
print("Quantized resulting shape is:",quantize_onset.shape)

File name is MusicDelta_Disco_Drum
Onset shape is (12478, 314)
Quantized resulting shape is: (12478,)


In [23]:
quantize_onset.shape

(12478,)

# Prepare splits for minibatch training

In [24]:
train = DataSequence([onsets_db.x[i] for i in train_files_indexes],
                     [onsets_db.annotations[i] for i in train_files_indexes],
                      batch_size=1, max_seq_length=60 * FPS)
                             
val = DataSequence([onsets_db.x[i] for i in validation_files_indexes],
                   [onsets_db.annotations[i] for i in validation_files_indexes],
                    batch_size=1, max_seq_length=60 * FPS)

In [25]:
train.batch_size

1

In [26]:
onset_idx = 1
x_example, y_example = train.x[onset_idx],train.y[onset_idx]
print(f"Example shape x is {x_example.shape}")
print(f"Example shape y is {y_example.shape}")

Example shape x is (11099, 314)
Example shape y is (11099,)


## I did it with n_epochs = 100 but 10 epochs is enough for showing the example

In [27]:
# few parameters
nn_params = {
    "learning_rate":0.01,
    "n_epochs":10, # TODO set this to 100
    "momentum":0.93
}



In [28]:
load_preexisinng_model = False
last_epoch_model_name = "" # enter last epoch  model name in case is needed
#
if load_preexisinng_model:
  current_model = os.path.join(basedir, last_epoch_model_name)
  print(f"[INFO] Loading model from {current_model}")
  model = keras.models.load_model(current_model)
else:
  print("[INFO] Creating model from scratch")
  model = Sequential()
  model.add(Masking(input_shape=(None, train[0][0].shape[-1]), mask_value=train.mask_value))
  model.add(Bidirectional(SimpleRNN(units=25, return_sequences=True)))
  model.add(Bidirectional(SimpleRNN(units=25, return_sequences=True)))
  model.add(Bidirectional(SimpleRNN(units=25, return_sequences=True)))
  model.add(Dense(units=1, activation='sigmoid'))

  model.compile(loss=keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.SGD(learning_rate=nn_params["learning_rate"],
                                                  clipvalue=5,
                                                  momentum=nn_params["momentum"]),
                metrics=['binary_accuracy'])

[INFO] Creating model from scratch


In [29]:
verbose=0
print(f"WE WILL BE WRITING CHECKPOINTS HERE:{basedir}")
mca = keras.callbacks.ModelCheckpoint(basedir + '/model_'+example_suffix+'_{epoch:02d}.h5',
                                      monitor='loss',
                                      save_best_only=False,
                                      save_weights_only=False,
                                      mode='auto',
                                      save_freq=8)
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   min_delta=1e-4,
                                   patience=20, 
                                   verbose=verbose)
tb = keras.callbacks.TensorBoard(log_dir=basedir + '/logs',
                                 write_graph=True,
                                 write_images=True)

WE WILL BE WRITING CHECKPOINTS HERE:/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/models_for_github


In [30]:
# change this booll in case you wanna retrain the model
train_model = True
if train_model:
  history = model.fit_generator(train,
                                steps_per_epoch=len(train),
                                epochs=nn_params["n_epochs"],
                                shuffle=True,
                                validation_data=val,
                                validation_steps=len(val),
                                callbacks=[mca, es, tb])
  model.save(basedir + f'/model_final_{example_suffix}.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/10
8/8 [==============================] - 330s 33s/step - loss: 0.3874 - binary_accuracy: 0.7638 - val_loss: 0.2281 - val_binary_accuracy: 0.9379
Epoch 2/10
8/8 [==============================] - 318s 44s/step - loss: 0.2012 - binary_accuracy: 0.9469 - val_loss: 0.2478 - val_binary_accuracy: 0.9379
Epoch 3/10
8/8 [==============================] - 304s 42s/step - loss: 0.1976 - binary_accuracy: 0.9469 - val_loss: 0.2216 - val_binary_accuracy: 0.9379
Epoch 4/10
8/8 [==============================] - 298s 34s/step - loss: 0.1843 - binary_accuracy: 0.9469 - val_loss: 0.2137 - val_binary_accuracy: 0.9379
Epoch 5/10
8/8 [==============================] - 307s 34s/step - loss: 0.1787 - binary_accuracy: 0.9469 - val_loss: 0.1956 - val_binary_accuracy: 0.9379
Epoch 6/10
8/8 [==============================] - 311s 44s/step - loss: 0.1655 - binary_accuracy: 0.9469 - val_loss: 0.1871 - val_binary_accuracy: 0.9379
Epoch 7/10
8/8 [==============================] - 308s 44s/step - loss: 0.15

In [31]:
os.listdir(basedir)

['onsets',
 'logs',
 'model_for_github_01.h5',
 'model_for_github_02.h5',
 'model_for_github_03.h5',
 'model_for_github_04.h5',
 'model_for_github_05.h5',
 'model_for_github_06.h5',
 'model_for_github_07.h5',
 'model_for_github_08.h5',
 'model_for_github_09.h5',
 'model_for_github_10.h5',
 'model_for_github.h5']

In [35]:
last_epoch_model_name = f"model_final_{example_suffix}.h5"
current_model_path = os.path.join(basedir, last_epoch_model_name)
model = keras.models.load_model(current_model_path)
model

# 4) Predict on the heldout set and save them into a pickle (so we can handle them later on the evals_. notebook) 

In [36]:
ANNOTATIONS_DIR_TEST = os.path.join(MUSIC_DIR,"annotations","class","test" )#/class/train"
annotation_files_all_test = glob.glob(ANNOTATIONS_DIR_TEST +"/*.txt")
audio_files_all_test = [search_correspondingpath_given_annotation(annotation, ONSET_PATH)  for annotation in annotation_files_all_test ]
# tomo el nombre base con el suffix
files_test = [os.path.basename(file) for file in audio_files_all_test]
#le saco el suffix
files_test = [file.split(".")[0] for file in files_test ]
onsets_db_test = Dataset(path = ONSET_PATH, files = files_test, annotation_files = annotation_files_all_test, audio_files= audio_files_all_test)


path2dump_preprocess_test = os.path.join(MUSIC_DIR,f"onset_db_test_{example_suffix}.pkl")
print("[INFO] Preprocessing files")
if not os.path.exists(path2dump_preprocess_test):
  #load the annotations
  print("[INFO] Loading the annotations")
  onsets_db_test.load_annoatations()
  print("[INFO] Pre processing the files")
  onsets_db_test.pre_process(pp)
  pickle.dump(onsets_db_test, open(path2dump_preprocess_test, 'wb'), protocol=2)
  print(f"Pickle dumped to: {path2dump_preprocess_test} ")
else:
  print(f"Loading previously dumped pickle from: {path2dump_preprocess_test}")
  onsets_db_test = pickle.load(open(path2dump_preprocess_test,'rb'))
print("[INFO] Preprocessing finished")

[INFO] Preprocessing files
[INFO] Loading the annotations
[INFO] Pre processing the files
Pickle dumped to: /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/onset_db_test_for_github.pkl 
[INFO] Preprocessing finished


In [37]:
onsets_db_test.audio_files

['/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_Hendrix_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_SwingJazz_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_FreeJazz_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_Beatles_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_Country1_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_SpeedMetal_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/audio/drum_only/test/MusicDelta_Punk_Drum.wav',
 '/content/drive/My Drive/Maestria DM y KDD/

In [38]:
# peak picking function
#     fps : float, optional
        #Frames per second used for conversion of timings.
rnn_peak_picking = madmom.features.onsets.OnsetPeakPickingProcessor(
        threshold=0.35, pre_max=0.01, post_max=0.01, smooth=0.07, combine=0.03, fps = FPS)

#predicted_song_idx = 0
n_songs2predict = len(onsets_db_test.audio_files)
predictions_dfs_list = list()
for predicted_song_idx in range(n_songs2predict):
  print(f"[INFO] Starting prediction for song_idx: {predicted_song_idx}")
  audio_path = onsets_db_test.audio_files[predicted_song_idx]
  # load song j 
  reshaped_onset = onsets_db_test.x[predicted_song_idx]
  # we will reshape with this; cahnge  dimensions from (K,M) to (1,K,M)
  reshaped_onset = reshaped_onset[np.newaxis,...]
  pred = model.predict(reshaped_onset)
  # change dimentions from (1,K,1) to (K,)
  pred = pred.squeeze()

  # now pass ; so you get onset_time
  final_onsets_song = rnn_peak_picking(pred)
  df_pred = pd.DataFrame(final_onsets_song, columns=["onset_time"])
  df_pred["audio_path"] = audio_path
  df_pred = df_pred[["onset_time","audio_path"]]
  predictions_dfs_list.append(df_pred)
df_predictions_rnn = pd.concat(predictions_dfs_list, axis = 0 )


[INFO] Starting prediction for song_idx: 0
[INFO] Starting prediction for song_idx: 1
[INFO] Starting prediction for song_idx: 2
[INFO] Starting prediction for song_idx: 3
[INFO] Starting prediction for song_idx: 4
[INFO] Starting prediction for song_idx: 5
[INFO] Starting prediction for song_idx: 6
[INFO] Starting prediction for song_idx: 7
[INFO] Starting prediction for song_idx: 8
[INFO] Starting prediction for song_idx: 9
[INFO] Starting prediction for song_idx: 10


In [39]:
predictions_onset_dataframe_path = os.path.join(basedir,f"model_predictions_rnn_fulltrained_{example_suffix}.pkl")
if not os.path.exists(predictions_onset_dataframe_path):
  df_predictions_rnn.to_pickle(predictions_onset_dataframe_path)
  print(f"Model predictions saved to {predictions_onset_dataframe_path}")
else:
  raise ValueError("You cannot overwrite the file; please use os.remove(filepath) first")


Model predictions saved to /content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/models_for_github/model_predictions_rnn_fulltrained_for_github.pkl


# We will load the saved predictions later for evaluating the onset detections combined with the recognition model

In [41]:
model_predictions_path_hardcoded = f'/content/drive/My Drive/Maestria DM y KDD/Especializacion tesis/MDBDrums/MDB Drums/models_for_github/model_predictions_rnn_fulltrained_{example_suffix}.pkl'
df_predictions_rnn = pd.read_pickle(model_predictions_path_hardcoded)
df_predictions_rnn

,onset_time,audio_path
0,0.02,/content/drive/My Drive/Maestria DM y KDD/Espe...
1,0.29,/content/drive/My Drive/Maestria DM y KDD/Espe...
2,0.57,/content/drive/My Drive/Maestria DM y KDD/Espe...
3,1.00,/content/drive/My Drive/Maestria DM y KDD/Espe...
4,1.38,/content/drive/My Drive/Maestria DM y KDD/Espe...
...,...,...
112,38.99,/content/drive/My Drive/Maestria DM y KDD/Espe...
113,39.26,/content/drive/My Drive/Maestria DM y KDD/Espe...
114,39.53,/content/drive/My Drive/Maestria DM y KDD/Espe...
115,39.81,/content/drive/My Drive/Maestria DM y KDD/Espe...
